# Premise

Let's make it clear that you're basically done with the entirety of the Pytorch Workflow (Except a few niche things from Pytorch 2.0)

This part isn't very related to Pytorch workflow, it's to create an user interface so other people can actually use your machine learning model

Since uh, we don't know how to do that, we are going to use some library to help us do it

Gradio, the exact same interface tool as WebUI Stable Diffusion

<img src="assets/Interface.png" alt="The WorkFlow" width="800">

The fancy name is called "model deployment", basically sending your model off into the real world and see how it performs out from this local notebook, there are a few things to consider though

## Where is the Model going to go?

Basically, How much computing power does it need?

- Running Locally on devices
- Running on Cloud servers

<img src="assets/Deployment_pro_con.png" alt="The WorkFlow" width="800">

Here are a few examples of places where models go to

<img src="assets/Tools_and_Places.png" alt="The WorkFlow" width="800">

## How is the model going to function?

Basically, How long should it take the model run?

- Running in Realtime
- Running in Batch Processing

``<img src="assets/Function_pro_con.png" alt="The WorkFlow" width="800">

# Training...?

Uh, in this part he again trains 2 models, an EffNetB2 and a ViT

I don't think it's necessary to create these models again, so I've just grabbed some previously trained ones from the before sections and start at the comparison between the 2 models (section 5.1)

Which is going to be EffnetB0 from Transfer Learning, and ViT from Paper Replication

# Comparing Model Performance

So the big idea for comparing model performance is

- speed
- accuracy

So we will iterate through all the data, see how accurate, and fast our EffnetB0 and ViT is

In [2]:
import pathlib
import torch

from PIL import Image
from timeit import default_timer as timer 
from tqdm.auto import tqdm
from typing import List, Dict

def pred_and_store(model, transform, class_names, paths, device):

    pred_list = []

    for path in tqdm(paths):
        pred_dict = {}

        #Get the sample path, and sample class name
        pred_dict["image_path"] = path
        class_name = path.parent.stem
        pred_dict["class_name"] = class_name

        #Start timer
        start_time = timer()

        #Open image path
        image = Image.open(path)

        #Transform the Image
        image = transform(image).unsqueeze(0).to(device)

        #Set up model for use
        model.to(device)
        model.eval()

        #Sending it through the model
        with torch.inference_mode():
            prediction_logits = model(image)
            prediction_probability_distribution = torch.softmax(prediction_logits, dim=1)
            prediction_label = torch.argmax(prediction_probability_distribution, dim=1)  
            prediction_class = class_names[prediction_label.cpu()] # The thing is, the list class_names is on the CPU and not GPU, so we have to send the label over to the CPU

            pred_dict["pred_prob"] = prediction_probability_distribution
            pred_dict["pred_class"] = prediction_class

        #End the timer
        end_time = timer()
        pred_dict["time_for_pred"] = round(end_time-start_time, 4)

        #Does the pred match the true label?
        pred_dict["correct"] = class_name == prediction_class

        #Add the dictionary to the list of predictions
        pred_list.append(pred_dict)
    
    #Return list of prediction dictionaries
    return pred_list

In [3]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

But first uh, we have to get the data

In [4]:
from pathlib import Path

# Setup path to data folder
data_path = Path("Data/")

# Setup Dirs
train_dir = data_path / "train"
test_dir = data_path / "test"

In [5]:
# Get all test data paths
print(f"Finding all filepaths ending with '.jpg' in directory: {test_dir}")
test_data_paths = list(Path(test_dir).glob("*/*.jpg"))

Finding all filepaths ending with '.jpg' in directory: Data\test


Load EffnetB0 and run the predictions

Though, this is actually quite tedious as we have modified the EffnetB0 by a bit, we need to make sure the model architecture matches the expected one by the saved weights

So we need to:

- Load a default pre-trained model from the architecture
- Make our tweaks
- Load our model from our pth file
- Load the Dataloaders for EffnetB0

In [5]:
import torchvision

from torch import nn

#Load the Pretrained Model
effnetb0_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
effnetb0_transforms = effnetb0_weights.transforms()
effnetb0 = torchvision.models.efficientnet_b0(weights=effnetb0_weights)

#Update the classifier part of the model
effnetb0.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=3, bias=True)).to(device)

# Load your custom model saved weights
model_path = 'models/pretrained_effnet_feature_extractor_pizza_steak_sushi.pth'
effnetb0.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [14]:
from going_modular import data_setup
train_dataloader_effnetb2, test_dataloader_effnetb2, class_names = data_setup.create_dataloaders(train_dir=train_dir, test_dir=test_dir, transform=effnetb0_transforms, batch_size=32)

In [10]:
effnetb0_test_pred_dicts = pred_and_store(model=effnetb0, transform=effnetb0_transforms, class_names=class_names, paths=test_data_paths, device="cpu")

  0%|          | 0/75 [00:00<?, ?it/s]

Load ViT and run the predictions

Yes, the same tedious procedure

In [11]:
import torchvision

from torch import nn

#Load the Pretrained Model
vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT
vit_transforms = vit_weights.transforms()
vit = torchvision.models.vit_b_16(weights=vit_weights).to(device)

#Update the classifier part of the model
vit.heads = nn.Linear(in_features=768, out_features=3).to(device)

# Load your custom model saved weights
model_path = 'models/pretrained_vit_feature_extractor_pizza_steak_sushi.pth'
vit.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [12]:
from going_modular import data_setup
train_dataloader_vit, test_dataloader_vit, class_names = data_setup.create_dataloaders(train_dir=train_dir, test_dir=test_dir, transform=vit_transforms, batch_size=32)

In [13]:
vit_test_pred_dicts = pred_and_store(paths=test_data_paths, model=vit, transform=vit_transforms, class_names=class_names, device="cpu")

  0%|          | 0/75 [00:00<?, ?it/s]

# Turn into Dataframes

In our comparison, we want to consider:

- how accurate is the model in the test
- how long did it take for the model in the test (average time per prediction)
- how many parameters is the model
- how large is the model

## Effnetb0

In [31]:
# Turn the test_pred_dicts into a DataFrame
import pandas as pd
effnetb0_test_pred_df = pd.DataFrame(effnetb0_test_pred_dicts)
effnetb0_test_pred_df.head(n=10)

,image_path,class_name,pred_prob,pred_class,time_for_pred,correct
0,Data\test\pizza\1152100.jpg,pizza,"[[tensor(0.9272), tensor(0.0543), tensor(0.018...",pizza,0.1068,True
1,Data\test\pizza\1503858.jpg,pizza,"[[tensor(0.7563), tensor(0.0170), tensor(0.226...",pizza,0.0391,True
2,Data\test\pizza\1687143.jpg,pizza,"[[tensor(0.8898), tensor(0.0580), tensor(0.052...",pizza,0.0335,True
3,Data\test\pizza\1925494.jpg,pizza,"[[tensor(0.8118), tensor(0.0774), tensor(0.110...",pizza,0.0356,True
4,Data\test\pizza\194643.jpg,pizza,"[[tensor(0.6921), tensor(0.0580), tensor(0.249...",pizza,0.0329,True
5,Data\test\pizza\195160.jpg,pizza,"[[tensor(0.7784), tensor(0.1469), tensor(0.074...",pizza,0.0305,True
6,Data\test\pizza\2003290.jpg,pizza,"[[tensor(0.7350), tensor(0.2230), tensor(0.042...",pizza,0.0311,True
7,Data\test\pizza\2019408.jpg,pizza,"[[tensor(0.8516), tensor(0.0319), tensor(0.116...",pizza,0.0314,True
8,Data\test\pizza\2111981.jpg,pizza,"[[tensor(0.9443), tensor(0.0114), tensor(0.044...",pizza,0.0604,True
9,Data\test\pizza\2124579.jpg,pizza,"[[tensor(0.9897), tensor(0.0019), tensor(0.008...",pizza,0.0302,True


In [32]:
# Check number of correct predictions
effnetb0_results = effnetb0_test_pred_df.correct.value_counts()
effnetb0_results_dict = effnetb0_results.to_dict()

# Find the average time per prediction 
effnetb0_average_time_per_pred = round(effnetb0_test_pred_df.time_for_pred.mean(), 4)

# Count number of parameters in EffNetB2
effnetb0_total_params = sum(torch.numel(param) for param in effnetb0.parameters())

# Get the model size in bytes then convert to megabytes
pretrained_effnetb0_model_size = Path("models/pretrained_effnet_feature_extractor_pizza_steak_sushi.pth").stat().st_size // (1024*1024) # division converts bytes to megabytes (roughly) 

In [33]:
# Create a dictionary with EffNetB2 statistics
effnetb2_stats = {"number_of_parameters": effnetb0_total_params,
                  "model_size (MB)": pretrained_effnetb0_model_size,
                  "pred_correct_results": effnetb0_results_dict.get(True, 0),
                  "pred_wrong_results": effnetb0_results_dict.get(False, 0),
                  "time_per_pred_cpu": effnetb0_average_time_per_pred}

## ViT

In [34]:
# Turn vit_test_pred_dicts into a DataFrame
import pandas as pd
vit_test_pred_df = pd.DataFrame(vit_test_pred_dicts)
vit_test_pred_df.head(n=10)

,image_path,class_name,pred_prob,pred_class,time_for_pred,correct
0,Data\test\pizza\1152100.jpg,pizza,"[[tensor(0.9949), tensor(0.0020), tensor(0.003...",pizza,0.4680,True
1,Data\test\pizza\1503858.jpg,pizza,"[[tensor(0.9927), tensor(0.0020), tensor(0.005...",pizza,0.1228,True
2,Data\test\pizza\1687143.jpg,pizza,"[[tensor(0.9974), tensor(0.0016), tensor(0.001...",pizza,0.0852,True
3,Data\test\pizza\1925494.jpg,pizza,"[[tensor(0.9902), tensor(0.0029), tensor(0.006...",pizza,0.0870,True
4,Data\test\pizza\194643.jpg,pizza,"[[tensor(0.9970), tensor(0.0020), tensor(0.001...",pizza,0.0873,True
5,Data\test\pizza\195160.jpg,pizza,"[[tensor(0.9985), tensor(0.0010), tensor(0.000...",pizza,0.0861,True
6,Data\test\pizza\2003290.jpg,pizza,"[[tensor(0.9978), tensor(0.0011), tensor(0.001...",pizza,0.1099,True
7,Data\test\pizza\2019408.jpg,pizza,"[[tensor(0.9946), tensor(0.0035), tensor(0.001...",pizza,0.0930,True
8,Data\test\pizza\2111981.jpg,pizza,"[[tensor(0.9974), tensor(0.0012), tensor(0.001...",pizza,0.0899,True
9,Data\test\pizza\2124579.jpg,pizza,"[[tensor(0.9951), tensor(0.0027), tensor(0.002...",pizza,0.1220,True


In [35]:
# Check number of correct predictions
vit_results = vit_test_pred_df.correct.value_counts()
vit_results_dict = vit_results.to_dict()

# Find the average time per prediction 
vit_average_time_per_pred = round(vit_test_pred_df.time_for_pred.mean(), 4)

# Count number of parameters in EffNetB2
vit_total_params = sum(torch.numel(param) for param in vit.parameters())

# Get the model size in bytes then convert to megabytes
pretrained_vit_model_size = Path("models/pretrained_effnet_feature_extractor_pizza_steak_sushi.pth").stat().st_size // (1024*1024) # division converts bytes to megabytes (roughly) 

In [36]:
# Create a dictionary with EffNetB2 statistics
vit_stats = {"number_of_parameters": vit_total_params,
            "model_size (MB)": pretrained_vit_model_size,
            "pred_correct_results": vit_results_dict.get(True, 0),
            "pred_wrong_results": vit_results_dict.get(False, 0),
            "time_per_pred_cpu": vit_average_time_per_pred}

## Compare models

Let's make a new dataframe from our statistics dictionaries, so this comparison can look a lot more straight forward

In [40]:
# Turn stat dictionaries into DataFrame
df = pd.DataFrame([effnetb2_stats, vit_stats])

# Add column for model names
df["model"] = ["EffNetB0", "ViT"]

# get a list of columns
cols = list(df)

# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index("model")))
df = df.loc[:, cols]

df

,model,number_of_parameters,model_size (MB),pred_correct_results,pred_wrong_results,time_per_pred_cpu
0,EffNetB2,4011391,15,65,10,0.0354
1,ViT,85800963,15,69,6,0.1109


Some takeaways are:

- ViT is a lot larger, and more accurate than EffNetB0
- On Average, EffnetB0 is a lot faster than Vit

# Deployment with Gradio

so, we've choosen EffnetB0 for it's faster performance, and now we are going to deploy it using Gradio 

From their own words <b>"Gradio is the fastest way to demo your machine learning model with a friendly web interface so that anyone can use it, anywhere!"</b>

So many libraries, just... so many

<img src="assets/Gradio.png" alt="The WorkFlow" width="800">

This will be what we will create towards the end

<img src="assets/FoodVisionMini.png" alt="The WorkFlow" width="800">


The way Gradio creates an interface is by using the function gradio.Interface(fn, inputs, outputs):

Where, fn is a Python function to map the inputs to the outputs

So we need to create an interface that allows inputs -> ML model -> outputs

For this use case, we have images of food -> EffNetB2 -> prediction of food

## ML Model Prediction Function

this will be a similar function to pred_and_store(), but we just need to make a prediction on a single image with EffNetB0

input: image -> transform -> predict with EffNetB2 -> output: pred, pred prob, time taken

In [7]:
# Put EffNetB0 on CPU, as running it on a GPU in hugging face costs money
effnetb0.to("cpu");

In [8]:
from typing import Tuple, Dict

def effnetb0_predict(img) -> Tuple[Dict, float]:
    """
    Transforms and performs a prediction on img and returns prediction and time taken.
    """
    # Start the timer
    start_time = timer()
    
    # Transform the target image and add a batch dimension
    img = effnetb0_transforms(img).unsqueeze(0)
    
    # Put model into evaluation mode and turn on inference mode
    effnetb0.eval()
    with torch.inference_mode():
        # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
        pred_probs = torch.softmax(effnetb0(img), dim=1)

    # Initialize an empty dictionary
    pred_labels_and_probs = {}

    # Loop over the indices of class_names, and create a prediction label and prediction probability dictionary for each prediction class (this is the required format for Gradio's output parameter)
    for i in range(len(class_names)):
        # Use the class name as the key and the corresponding predicted probability as the value
        pred_labels_and_probs[class_names[i]] = float(pred_probs[0][i])

    
    # Calculate the prediction time
    end_time = timer()
    pred_time = round(end_time - start_time, 4)
    
    # Return the prediction dictionary and prediction time 
    return pred_labels_and_probs, pred_time

## Creating Examples of Predictions

This is a list of example images, so people can directly try the model out with these images to see how our model will perform on food images

In [19]:
import random

# So each time we run this we get the same results
random.seed(25)

# Create a list of example inputs to our Gradio demo
examples_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=3)]
examples_list

[['Data\\test\\sushi\\1600999.jpg'],
 ['Data\\test\\pizza\\1503858.jpg'],
 ['Data\\test\\steak\\1285886.jpg']]

## Building Gradio Interface

We are going to input the following parameters to the gradio.Interface() class:

- fn: a function that maps inputs to outputs, in this case `effnetb0_predict`
- inputs: the input to the interface which get passed to the function, in this case `image`
- outputs: the output of the function which gets passed to the interface, in this case `label` and `time taken` (a number)
- examples: a list a of examples to showcase the model
- title: a title of the model
- description: a description of the model
- article: a reference note of the model

This will return a local url which you can open

In [15]:
import gradio

title = "FoodVision Mini 🍕🥩🍣"
description = "An EfficientNetB2 feature extractor computer vision model to classify images of food as pizza, steak or sushi."
article = "Created at (https://www.learnpytorch.io/09_pytorch_model_deployment/)."

demo = gradio.Interface(fn=effnetb0_predict, 
                        inputs="image", 
                        outputs=[gradio.Label(num_top_classes=3, label="Predictions"), gradio.Number(label="Prediction time (s)")],
                        examples=examples_list, 
                        title=title,
                        description=description,
                        article=article)

# Launch the demo!
demo.launch(debug=False, # print errors locally?
            share=False) # generate a publicly shareable URL?

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Here is a random image found online of a picture of steak, let's see if our model get's it correct

<img src="assets/Juicy_Steak.jpg" alt="The WorkFlow" width="500">

Yes it does, with a 68% guess of steak

<img src="assets/Gradio_Example.png" alt="The WorkFlow" width="1000">



And now, our machine learning models can be used by anyone through just a simple interface without any touching of code

Of course, there's more detail to dive into for how you want specific layouts to look like etc in Gradio, but that you can do so in your own time for your own projects

# Upload to Hugging Face Spaces

you can imagine this part as "the model is done, let's throw it to a platform where everyone can see, use, and download"

Hugging Face is basically the equivalent of Github for machine learning (It tries to be)

If having a good GitHub portfolio showcases your coding abilities, having a good Hugging Face portfolio can showcase your machine learning abilities.

<b>There are many other places we could upload and host our Gradio app such as, Google Cloud, AWS (Amazon Web Services) or other cloud vendors

however, we're going to use Hugging Face Spaces due to the ease of use and wide adoption by the machine learning community.</b>

## Structure

Everything must go under a single directory, this is an example of the final structure

```
demos/
└── foodvision_mini/
    ├── pretrained_effnet_feature_extractor_pizza_steak_sushi.pth
    ├── app.py
    ├── examples/
    │   ├── example_1.jpg
    │   ├── example_2.jpg
    │   └── example_3.jpg
    ├── model.py
    └── requirements.txt
```

- pretrained_effnet_feature_extractor is our trained Pytorch Model
- app.py is the gradio app
- examples contain example images for the gradio app
- model.py is the definition of our model, transforms, etc
- requirements.txt explains the dependencies to run our app such as torch, torchvision and gradio

## Creating Structure in Code

Create demos folder, and foodvision_mini folder

In [14]:
import shutil
from pathlib import Path

foodvision_mini_path = Path("demos/foodvision_mini")
foodvision_mini_path.mkdir(parents=True,exist_ok=True)

Creating examples folder, and examples

In [20]:
# Create an examples directory
examples_path = foodvision_mini_path / "examples"
examples_path.mkdir(parents=True,exist_ok=True)

# Collect three random test dataset image paths
example_list_path = [Path(path_string[0]) for path_string in examples_list]

# Copy the three random images to the examples directorys
for example_path in example_list_path:
    destination_path = examples_path / example_path.name
    shutil.copy2(src=example_path, dst=destination_path)

Copying effnetb0 over to demos folder

In [26]:
# Create a source path for our target model
effnetb0_path = "models/pretrained_effnet_feature_extractor_pizza_steak_sushi.pth"

# Create a destination path for our target model
effnetb0_destination_path = foodvision_mini_path /  "pretrained_effnet_feature_extractor_pizza_steak_sushi.pth"

# Copy the model over
shutil.copy2(src=effnetb0_path, dst=effnetb0_destination_path);

we need a way to instantiate a model, basically load the architecture and transforms

create that and put it as a python script, model.py

In [29]:
%%writefile demos/foodvision_mini/model.py
import torch
import torchvision

from torch import nn


def create_effnetb0_model(seed:int=42, 
                          num_classes:int=3):

    """
    Creates an EfficientNetB0 feature extractor model and transforms.

    Args:
        num_classes (int, optional): number of classes in the classifier head. 
            Defaults to 3.
        seed (int, optional): random seed value. Defaults to 42.

    Returns:
        model (torch.nn.Module): EffNet0 feature extractor model. 
        transforms (torchvision.transforms): EffNetB0 image transforms.
    """

    #Load the Pretrained Model
    effnetb0_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    effnetb0_transforms = effnetb0_weights.transforms()
    model = torchvision.models.efficientnet_b0(weights=effnetb0_weights)

    #Update the classifier part of the model
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features=1280, out_features=num_classes, bias=True))

    #Freeze all layers as we are not training it (this speeds up computation)
    for param in model.parameters():
        param.requires_grad = False

    return model, effnetb0_transforms

Overwriting demos/foodvision_mini/model.py


Finally, creating app.py, which loads our model from saved weights, and creates a Gradio Interface

We call it app.py because by default when you create a HuggingFace Space, it looks for a file called app.py to run and host

In [31]:
%%writefile demos/foodvision_mini/app.py



### 1. Imports and class names setup ### 

import gradio
import os
import torch

from model import create_effnetb0_model
from timeit import default_timer as timer
from typing import Tuple, Dict



### 2. Model and transforms preparation ###

# Setup class names
class_names = ["pizza", "steak", "sushi"]

# Create EffNetB2 model
effnetb0, effnetb0_transforms = create_effnetb0_model(num_classes=len(class_names))

# Load saved weights to the CPU
effnetb0.load_state_dict(torch.load(f="pretrained_effnet_feature_extractor_pizza_steak_sushi.pth", map_location=torch.device("cpu"),))



### 3. Predict function ###

# Create predict function
def effnetb0_predict(img) -> Tuple[Dict, float]:
    """
    Transforms and performs a prediction on img and returns prediction and time taken.
    """
    # Start the timer
    start_time = timer()
    
    # Transform the target image and add a batch dimension
    img = effnetb0_transforms(img).unsqueeze(0)
    
    # Put model into evaluation mode and turn on inference mode
    effnetb0.eval()
    with torch.inference_mode():
        # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
        pred_probs = torch.softmax(effnetb0(img), dim=1)

    # Initialize an empty dictionary
    pred_labels_and_probs = {}

    # Loop over the indices of class_names, and create a prediction label and prediction probability dictionary for each prediction class (this is the required format for Gradio's output parameter)
    for i in range(len(class_names)):
        # Use the class name as the key and the corresponding predicted probability as the value
        pred_labels_and_probs[class_names[i]] = float(pred_probs[0][i])

    
    # Calculate the prediction time
    end_time = timer()
    pred_time = round(end_time - start_time, 4)
    
    # Return the prediction dictionary and prediction time 
    return pred_labels_and_probs, pred_time



### 4. Gradio app ###

# Create title, description and article strings
title = "FoodVision Mini 🍕🥩🍣"
description = "An EfficientNetB0 feature extractor computer vision model to classify images of food as pizza, steak or sushi."
article = "Created at (https://www.learnpytorch.io/09_pytorch_model_deployment/)."

# Create examples list from "examples/" directory
example_list = [["examples/" + example] for example in os.listdir("examples")]

demo = gradio.Interface(fn=effnetb0_predict, 
                        inputs="image", 
                        outputs=[gradio.Label(num_top_classes=3, label="Predictions"), gradio.Number(label="Prediction time (s)")],
                        examples=example_list, 
                        title=title,
                        description=description,
                        article=article)

# Launch the demo!
demo.launch()

Overwriting demos/foodvision_mini/app.py


Creating a minimum_requirements.txt file, basically what you need to run this machine learning model demo

In [32]:
%%writefile demos/foodvision_mini/minimum_requirements.txt
torch==1.12.0
torchvision==0.13.0
gradio==3.1.4

Writing demos/foodvision_mini/minimum_requirements.txt


packaging it all up into a zip for upload

- base_name is the final name
- format is the format which we want the archive to be in
- root_dir is the directory which we create the archive
- base_dir is the directory which we output the archive

In [35]:
shutil.make_archive(base_name="demos_ready_for_hugging_face", format="zip", root_dir="demos", base_dir="")

'd:\\1. Python (AI)\\1. Tutorials\\4. Machine Learning Libraries\\1. Pytorch\\10. Model_Deployment\\../data/demos_ready_for_hugging_face.zip'

## Upload to Hugging Face

obviously, we won't really do this, but we will still see some ways on how to do so

- Uploading using Hugging Face Hub python library
- Uploading via the Hugging Face Web interface
- Upload via Command Line of Git

Then he goes into the specifics of creating an account, selecting a bunch of things, how to do Git on Command Line... I haven't touched a single piece of git yet

So nah  ¯ \ _ (ツ) _ / ¯ , we'll call it a day here

But, we will see an embedded, online FoodVision Mini Gradio demo into our notebook as an iframe with IPython.display.IFrame

In [42]:
# IPython is a library to help make Python interactive
from IPython.display import IFrame

# Embed FoodVision Mini Gradio demo
IFrame(src="https://hf.space/embed/mrdbourke/foodvision_mini/+", width=1300, height=800)

# Food Vision Big?

He then proceeds to build a BIG Food Vision Model and uploads it to hugging face, which predicts from not 3 classes, but 101

Uh... no thank you again, I really feel fine stopping here, and moving on to my own projects

which, this is the end of pytorch's basics (except addition niche things in pytorch 2.0)